In [3]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.09.22 insert 에러 추가수정 완료 - 31140 데이터가 대부분 없음
cursor = mydb.cursor(prepared=True)
citycode = '31150'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20210922 | location : 31140 | cnt : 1 | start GGB200000175 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt : 2 | start GGB200000193 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt : 3 | start GGB200000299 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt : 4 | start GGB200000310 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt : 5 | start GGB200000311 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt : 6 | start GGB223000085 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt : 7 | start GGB223000100 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt : 8 | start GGB223000130 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt : 9 | start GGB223000132 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt : 10 | start GGB234000324 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt : 11 | start GGB223000019 public_busroute_info
조회날짜 : 20210922 | location : 31140 | cnt 

location 31140 | GGB234001163 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 78 | start GGB208000027 public_busroute_info
location 31140 | GGB208000027 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 79 | start GGB226000022 public_busroute_info
location 31140 | GGB226000022 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 80 | start GGB227000002 public_busroute_info
location 31140 | GGB227000002 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 81 | start GGB227000005 public_busroute_info
location 31140 | GGB227000005 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 82 | start GGB227000017 public_busroute_info
location 31140 | GGB227000017 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 83 | start GGB227000039 public_busroute_info
location 31140 | GGB227000039 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 84 | start GGB227000045 public_busroute_info
location 31140 | GGB227000045 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 85 | start GG

location 31140 | GGB228000165 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 143 | start GGB228000176 public_busroute_info
location 31140 | GGB228000176 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 144 | start GGB228000177 public_busroute_info
location 31140 | GGB228000177 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 145 | start GGB228000181 public_busroute_info
location 31140 | GGB228000181 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 146 | start GGB228000190 public_busroute_info
location 31140 | GGB228000190 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 147 | start GGB228000200 public_busroute_info
location 31140 | GGB228000200 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 148 | start GGB228000203 public_busroute_info
location 31140 | GGB228000203 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 149 | start GGB228000214 public_busroute_info
location 31140 | GGB228000214 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 150 | 

location 31140 | GGB228000174 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 209 | start GGB228000175 public_busroute_info
location 31140 | GGB228000175 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 210 | start GGB228000184 public_busroute_info
location 31140 | GGB228000184 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 211 | start GGB228000197 public_busroute_info
location 31140 | GGB228000197 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 212 | start GGB228000209 public_busroute_info
location 31140 | GGB228000209 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 213 | start GGB228000249 public_busroute_info
location 31140 | GGB228000249 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 214 | start GGB228000001 public_busroute_info
location 31140 | GGB228000001 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 215 | start GGB228000010 public_busroute_info
location 31140 | GGB228000010 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 216 | 

location 31140 | GGB229000140 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 276 | start GGB229000236 public_busroute_info
location 31140 | GGB229000236 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 277 | start GGB229000254 public_busroute_info
location 31140 | GGB229000254 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 278 | start GGB229000264 public_busroute_info
location 31140 | GGB229000264 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 279 | start GGB229000267 public_busroute_info
location 31140 | GGB229000267 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 280 | start GGB229000010 public_busroute_info
location 31140 | GGB229000010 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 281 | start GGB229000025 public_busroute_info
location 31140 | GGB229000025 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 282 | start GGB229000033 public_busroute_info
location 31140 | GGB229000033 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 283 | 

location 31140 | GGB234001063 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 342 | start GGB234001075 public_busroute_info
location 31140 | GGB234001075 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 343 | start GGB234001078 public_busroute_info
location 31140 | GGB234001078 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 344 | start GGB234001079 public_busroute_info
location 31140 | GGB234001079 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 345 | start GGB234001081 public_busroute_info
location 31140 | GGB234001081 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 346 | start GGB234001133 public_busroute_info
location 31140 | GGB234001133 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 347 | start GGB234001178 public_busroute_info
location 31140 | GGB234001178 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 348 | start GGB234001186 public_busroute_info
location 31140 | GGB234001186 데이터가 없습니다.
조회날짜 : 20210922 | location : 31140 | cnt : 349 | 